# Top 500 players in the world (!Need bigger dataset)


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import auc, roc_curve, confusion_matrix

pd.options.mode.chained_assignment = None  # default='warn'

In [20]:
df = pd.read_csv("data/players.csv", index_col=0)
df

,Name,Position,Age,Markey Value In Millions(£),Country,Club,Matches,Goals,Own Goals,Assists,Yellow Cards,Second Yellow Cards,Red Cards,Number Of Substitute In,Number Of Substitute Out
0,Kylian Mbappé,Centre-Forward,22,144.0,France,Paris Saint-Germain,16,7,0,11,3,0,0,0,8
1,Erling Haaland,Centre-Forward,21,135.0,Norway,Borussia Dortmund,10,13,0,4,1,0,0,0,1
2,Harry Kane,Centre-Forward,28,108.0,England,Tottenham Hotspur,16,7,0,2,2,0,0,2,2
3,Jack Grealish,Left Winger,26,90.0,England,Manchester City,15,2,0,3,1,0,0,2,8
4,Mohamed Salah,Right Winger,29,90.0,Egypt,Liverpool FC,15,15,0,6,1,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Giorgian de Arrascaeta,Attacking Midfield,27,16.2,Uruguay,Clube de Regatas do Flamengo,0,0,0,0,0,0,0,0,0
496,Ayoze Pérez,Second Striker,28,16.2,Spain,Leicester City,8,1,0,3,0,0,1,2,5
497,Alex Meret,Goalkeeper,24,16.2,Italy,SSC Napoli,5,0,0,0,0,0,0,0,0
498,Duje Caleta-Car,Centre-Back,25,16.2,Croatia,Olympique Marseille,8,0,0,0,2,0,0,0,2


In [22]:
df.rename(columns={'Markey Value In Millions(£)': 'Value'},
          inplace=True, errors='raise')

KeyError: "['Markey Value In Millions(£)'] not found in axis"

In [23]:
df.head()

,Name,Position,Age,Value,Country,Club,Matches,Goals,Own Goals,Assists,Yellow Cards,Second Yellow Cards,Red Cards,Number Of Substitute In,Number Of Substitute Out
0,Kylian Mbappé,Centre-Forward,22,144.0,France,Paris Saint-Germain,16,7,0,11,3,0,0,0,8
1,Erling Haaland,Centre-Forward,21,135.0,Norway,Borussia Dortmund,10,13,0,4,1,0,0,0,1
2,Harry Kane,Centre-Forward,28,108.0,England,Tottenham Hotspur,16,7,0,2,2,0,0,2,2
3,Jack Grealish,Left Winger,26,90.0,England,Manchester City,15,2,0,3,1,0,0,2,8
4,Mohamed Salah,Right Winger,29,90.0,Egypt,Liverpool FC,15,15,0,6,1,0,0,0,3


In [24]:
median = df['Value'].median()
df['is_cheap'] = df['Value']<median
df.head()

,Name,Position,Age,Value,Country,Club,Matches,Goals,Own Goals,Assists,Yellow Cards,Second Yellow Cards,Red Cards,Number Of Substitute In,Number Of Substitute Out,is_cheap
0,Kylian Mbappé,Centre-Forward,22,144.0,France,Paris Saint-Germain,16,7,0,11,3,0,0,0,8,False
1,Erling Haaland,Centre-Forward,21,135.0,Norway,Borussia Dortmund,10,13,0,4,1,0,0,0,1,False
2,Harry Kane,Centre-Forward,28,108.0,England,Tottenham Hotspur,16,7,0,2,2,0,0,2,2,False
3,Jack Grealish,Left Winger,26,90.0,England,Manchester City,15,2,0,3,1,0,0,2,8,False
4,Mohamed Salah,Right Winger,29,90.0,Egypt,Liverpool FC,15,15,0,6,1,0,0,0,3,False


In [29]:
X = pd.get_dummies(df, columns=['Position','Country','Club'])
X.head()

,Name,Age,Value,Matches,Goals,Own Goals,Assists,Yellow Cards,Second Yellow Cards,Red Cards,...,Club_UD Almería,Club_US Sassuolo,Club_Valencia CF,Club_VfB Stuttgart,Club_VfL Wolfsburg,Club_Villarreal CF,Club_Watford FC,Club_West Ham United,Club_Wolverhampton Wanderers,Club_Zenit St. Petersburg
0,Kylian Mbappé,22,144.0,16,7,0,11,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Erling Haaland,21,135.0,10,13,0,4,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harry Kane,28,108.0,16,7,0,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Jack Grealish,26,90.0,15,2,0,3,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Mohamed Salah,29,90.0,15,15,0,6,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
X.drop(['Name', 'Value', 'is_cheap'], axis = 1, inplace = True)
X.head()

,Age,Matches,Goals,Own Goals,Assists,Yellow Cards,Second Yellow Cards,Red Cards,Number Of Substitute In,Number Of Substitute Out,...,Club_UD Almería,Club_US Sassuolo,Club_Valencia CF,Club_VfB Stuttgart,Club_VfL Wolfsburg,Club_Villarreal CF,Club_Watford FC,Club_West Ham United,Club_Wolverhampton Wanderers,Club_Zenit St. Petersburg
0,22,16,7,0,11,3,0,0,0,8,...,0,0,0,0,0,0,0,0,0,0
1,21,10,13,0,4,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,28,16,7,0,2,2,0,0,2,2,...,0,0,0,0,0,0,0,0,0,0
3,26,15,2,0,3,1,0,0,2,8,...,0,0,0,0,0,0,0,0,0,0
4,29,15,15,0,6,1,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0


In [32]:
y = df['is_cheap']

In [33]:
# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(
             X, y, test_size = 0.33, random_state=42)

In [34]:
k = 36 
knn = KNeighborsClassifier(n_neighbors=k)

In [35]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=36)

In [37]:
# Predict on dataset which model has not seen before
print(y_test)
print(knn.predict(X_test))
print(knn.score(X_test, y_test))

y_pred = knn.predict(X_test)

361     True
73     False
374     True
155    False
104    False
       ...  
110    False
318     True
428     True
29     False
437     True
Name: is_cheap, Length: 165, dtype: bool
[False False  True  True  True  True False False False False False  True
 False False False False  True False  True False False False False False
 False False  True False False False False False False False False False
 False False  True False False False False False  True  True False False
 False False False  True  True False  True  True False  True  True False
  True  True  True False  True False False False False False  True False
 False  True False False False False False  True  True False  True  True
 False False  True  True  True  True False False False False False False
  True False False False  True  True  True False False False False False
 False  True  True  True False False  True False False False  True False
  True False False False False  True  True False False False False False
 False False 

In [38]:
fpr, tpr, _ = roc_curve(y_test, y_pred, pos_label=True)

In [39]:
auc(fpr,tpr)

0.4769649690903739